In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
def get_html(url):
    r = requests.get(url)
    return r.text
def get_soup(html):
    return BeautifulSoup(html, 'html.parser')

In [56]:
url = 'https://www.marketbeat.com/stocks/NASDAQ/MSFT/price-target/?MostRecent=0'
soup = get_soup(get_html(url))

In [57]:
df = pd.read_html(str(soup.find('table', {'class': 'scroll-table sort-table'})))[0]

In [58]:
df.head()

,Date,Brokerage,Action,Rating,Price Target,Impact on Share Price,Details
0,3/16/2020,Wedbush,Reiterated Rating,Buy,$210.00,Medium,NaN
1,3/11/2020,Jefferies Financial Group,Reiterated Rating,Buy,$190.00,Medium,NaN
2,3/2/2020,Oppenheimer,Upgrade,Buy,NaN,High,NaN
3,2/27/2020,Deutsche Bank,Reiterated Rating,Buy,NaN,High,NaN
4,2/27/2020,Citigroup,Reiterated Rating,Hold,$174.00,High,NaN


In [71]:
def clean_price_target(inp_str):
    if '➝' in inp_str and len(inp_str) > 1:
        return float(inp_str.split('➝')[1].strip()[1:])
    elif inp_str == '➝':
        return 0.0
    else:
        return float(inp_str[1:])

In [75]:
df.Date = df['Date'].astype('datetime64')

df['Price Target'].fillna('00', inplace=True)

df['Price Target'] = df['Price Target'].apply(clean_price_target)

In [106]:
INS_ANLYST_SELL = df[df.Rating == 'Sell']\
                    .set_index('Date').groupby(pd.Grouper(freq='M'))['Price Target'].count()

INS_ANLYST_HOLD = df[df.Rating == 'Hold']\
                    .set_index('Date').groupby(pd.Grouper(freq='M'))['Price Target'].count()

INS_ANLYST_BUY = df[df.Rating == 'Buy']\
                    .set_index('Date').groupby(pd.Grouper(freq='M'))['Price Target'].count()

In [119]:
AMT_ANLYST_SELL = df[df.Rating == 'Sell']\
                    .set_index('Date').groupby(pd.Grouper(freq='M'))['Price Target'].mean()

AMT_ANLYST_HOLD = df[df.Rating == 'Hold']\
                    .set_index('Date').groupby(pd.Grouper(freq='M'))['Price Target'].mean()

AMT_ANLYST_BUY = df[df.Rating == 'Buy']\
                    .set_index('Date').groupby(pd.Grouper(freq='M'))['Price Target'].mean()